In [ ]:
from azure.quantum import Workspace
workspace = Workspace (
   resource_id = "/subscriptions/94f5e81d-8575-4e4a-aa46-55606a36edac/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/Machine-Learning-jUPYTER-Notebook",
   location = "eastus"
)


# Natural Language Processing (NLP)
 This is the computing that deals with communication between humans and computers examples are speelcheck or autocomplete.
# Recurrent  Neural Networks.
The neural network that we will use will be much capabal of prosessing seuential data such as text or characters called a recurrent Neural Network (RNN for short)
we will learnhow to use a RNN to do the following
- sentiment Analysis( this is basically finding out how positive or negative a peice of information is)
- Character Generation
# 
RNNs come in very complex form so we will learn how they work  and the problems they are suited for.

## Sequence Data
This is unlike image data that has width height and channels, text data that we are going to use will be special. in that  the textual data contains many words that follow ina very specific and meaningful order we need to keep track of each word and when it occurs in the data.

### Encoding Text
As we know  machine learning models and neural networks dont take raw data as input.
Howe we change text into numerical data methods:
#
- Bag of words (only works on simple tasks )

##### Example of code for the Bag of words how it works

In [3]:
vocab = {} # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
    global word_encoding
    words = text.lower().split(" ")# create a lits of all of the words in the text
    bag={}
    for word in words:
        if word in vocab:
            encoding = vocab[word] # get encoding from vocab
        else:
            vocab[word] = word_encoding
            encoding= word_encoding
            word_encoding += 1
        
        if encoding in bag:
            bag[encoding] += 1
        else:
            bag[encoding] =1
    
    return bag
text = "This is a test to see if this test will work is is test a a" 
bag = bag_of_words(text)
print(bag)
print(vocab)



{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


## WORD EMBEDING
This is a vectorized representation of words in a given documment that places words with similar meanings near each other.

## Recurrent Neural Networks (RNN's)
This has an internal loop in it, in the sense that it does not procsess the data all at once.
# 
It has a recurrent layer loop

## LSTM (Long Short-Term Memory)
this layer works like a LNN but it adds another way to access inputs from any timestep in the past.
# 



# Sentiment Analysis (this is to determine if a block of words is positive ore negative)
We are going to be using dataset of movie review

The movie review dataset contains 25000 reveiws from IMDB where each one is already processed and has a label as either positive or negative.  each reveiw is encoded by intergers that represents how common a word is in the entire dataset# For example a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.

#### Always make sure tensorflow is installed if not use this code
!pip install tensorflow  # if your in a notebook

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np 
VOCAB_SIZE = 88584

MAXLEN =250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

2023-08-13 18:28:42.442037: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 18:28:42.517062: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-13 18:28:42.521871: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 18:28:44.772220: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


17464789/17464789 [==============================] - 0s 0us/step


In [3]:
#Lets look at one review
train_data[0]
len(train_data[0])

218

## More Preprocessing
Remeber we cannot feed Neural Networks or machine learning models with differnt sizes of data. we have to make the data same size.
# 
Example ower data is not of the same lengths. therefor we must make each review the same length to do this we will follow the procedure below:

- If the review is greater than 250 words then trim off the extra words
- If the review is less than 250 words add the necessary amount 0's to make is equal to 250 words

Luckily for us keras has a function that can do this for us

In [4]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

## Creating the Model
Now we will use a word embedding layer as first layer in our model and add a LSTM layers afterwards that feeds into a dense node to get our predicted sentiment.
# 
32 stands for the output dimensions of the vectores generated by the embedding layers. we can change this value if we want to.

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),              # sigmoid squishes the values to be between 0 and 1
    tf.keras.layers.Dense(1, activation = 'sigmoid') # here we are trying to predict if the setiment is true or false we use the...
                                                    # Dense 0 - 1 so that if the answer is greater than 0.5 then we can rate that...
                                                    # a positive review and vise verser 
])
#the model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_7 (LSTM)               (None, 32)                8320      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Training
Now its time to compile and train our model

In [17]:
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['acc']) # this is compiling the model
history = model.fit(train_data, train_labels, epochs = 10, validation_split = 0.2) #( here we will use only 0.2 (20%)of the data tpo evaluate the model)
# binary_crossentropy this tells us how far we are from the correct probability coz we are predicting either zero or one
# getting a high accureacy basically tells us that we are either using not enough data, or change the model.

Epoch 1/10
625/625 [==============================] - 98s 153ms/step - loss: 0.4482 - acc: 0.7818 - val_loss: 0.2932 - val_acc: 0.8784
Epoch 2/10
625/625 [==============================] - 88s 141ms/step - loss: 0.2617 - acc: 0.9022 - val_loss: 0.3067 - val_acc: 0.8782
Epoch 3/10
625/625 [==============================] - 84s 135ms/step - loss: 0.2006 - acc: 0.9262 - val_loss: 0.2878 - val_acc: 0.8840
Epoch 4/10
625/625 [==============================] - 84s 134ms/step - loss: 0.1645 - acc: 0.9435 - val_loss: 0.3552 - val_acc: 0.8750
Epoch 5/10
625/625 [==============================] - 87s 139ms/step - loss: 0.1360 - acc: 0.9528 - val_loss: 0.3352 - val_acc: 0.8866
Epoch 6/10
625/625 [==============================] - 84s 134ms/step - loss: 0.1139 - acc: 0.9616 - val_loss: 0.4590 - val_acc: 

##### And well evaluate the model on our training data  to see how weel it performs

In [18]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 37s 47ms/step - loss: 0.4760 - acc: 0.8619
[0.4759834408760071, 0.8619199991226196]


#### So we are scorring somewhere in the mid-high 80s not bad for a smple recurrent network

### Making Predictions
Now by using our network lets make predictions on our own reviews
# 
Since our review are encoded we will need to convert any review that we right into that form so the network can understand it. 
# 
To do that well load the encodings from the dataset and use them to encode our own data

In [22]:
word_index =  imdb.get_word_index()
def encode_text(text):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word  in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [23]:
# While we are at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text= ""
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num] +  " "
    return text[:-1]
print(decode_integers(encoded))

that movie was just amazing so amazing


In [28]:
#NOW time to make a prediction
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250)) # this is becouse our model expects numbers from 1 to 250
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])

positive_review = "That movie was so awesome! I really Love it and would watch it again becouse it was amazingly great"
predict(positive_review)

1/1 [==============================] - 0s 78ms/step
[0.53327113]


In [29]:
negative_review = " that movie sucked. I hated it and wouldn't watch it again. was one of the worst things i have ever watched"
predict(negative_review)

1/1 [==============================] - 0s 65ms/step
[0.8039897]


### Go to next notebook 1.1 to create a model with RNN that generates / predicts next characters 